In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __builtin__ import any as b_any

import math
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
import numpy as np
from PIL import Image
from alexnet import AlexNet

import tensorflow as tf

import configuration
import inference_wrapper
import sys
sys.path.insert(0, 'im2txt/inference_utils')
import caption_generator
import vocabulary

In [2]:
np.random.seed(2)

current_dir = os.getcwd()
image_dir = os.path.join(current_dir, 'Jmemes')


checkpoint_path="trainlogIncNEW"
vocab_file ="vocab4.txt"
input_files ="Jmemes"         

tf.logging.set_verbosity(tf.logging.INFO)


In [3]:
#configuration
config = configuration.ModelConfig()

g = tf.Graph()
with g.as_default():
    model = inference_wrapper.InferenceWrapper()
    restore_fn = model.build_graph_from_config(config,
                                               checkpoint_path)
g.finalize()


INFO:tensorflow:Building model.
About to decide if splitting
new_h Tensor("lstm/basic_lstm_cell/Mul_2:0", shape=(1, 512), dtype=float32)
new_state LSTMStateTuple(c=<tf.Tensor 'lstm/basic_lstm_cell/Add_1:0' shape=(1, 512) dtype=float32>, h=<tf.Tensor 'lstm/basic_lstm_cell/Mul_2:0' shape=(1, 512) dtype=float32>)
About to decide if splitting
It's splitting
new_h Tensor("lstm/basic_lstm_cell/Mul_5:0", shape=(1, 2, 512), dtype=float32)
new_state LSTMStateTuple(c=<tf.Tensor 'lstm/basic_lstm_cell/Add_3:0' shape=(1, 2, 512) dtype=float32>, h=<tf.Tensor 'lstm/basic_lstm_cell/Mul_5:0' shape=(1, 2, 512) dtype=float32>)
lstm_outputs Tensor("lstm/basic_lstm_cell/Mul_5:0", shape=(1, 2, 512), dtype=float32)
state_tuple LSTMStateTuple(c=<tf.Tensor 'lstm/basic_lstm_cell/Add_3:0' shape=(1, 2, 512) dtype=float32>, h=<tf.Tensor 'lstm/basic_lstm_cell/Mul_5:0' shape=(1, 2, 512) dtype=float32>)
BUILDING DENSE
MATMUL(TENSORDOT) w/ SPLITTING


In [4]:
vocab = vocabulary.Vocabulary(vocab_file)

with open('Captions.txt','r') as f:
    data_captions = f.readlines()
data_captions = [s.lower() for s in data_captions]
  
with open('ordered_memes.txt','r') as f:
    ordered_memes = f.readlines()
ordered_memes = [meme.replace('\n','') for meme in ordered_memes]

INFO:tensorflow:Initializing vocabulary from file: vocab4.txt
INFO:tensorflow:Created vocabulary with 38521 words


In [5]:
np.random.seed(2)
#convert jpg image(s) into iamge representations using alexnet:
filenames = [os.path.join(image_dir, f) for f in os.listdir(input_files) if f.endswith('.jpg')]
#filenames = [os.path.join(image_dir, f) for f in ordered_memes[150:160] + ['TutorPP.jpg']]
print(filenames)
tf.logging.info("Running caption generation on %d files matching %s",len(filenames), input_files)
with tf.Session(graph=g) as sess:
    # Load the model from checkpoint.
    restore_fn(sess)
    
    # Prepare the caption generator. Here we are implicitly using the default
    # beam search parameters. See caption_generator.py for a description of the
    # available beam search parameters.
    generator = caption_generator.CaptionGenerator(model, vocab, beam_size=config.beam_size)
    num_in_data_total = 0
    num_captions = 0
    for i,filename in enumerate(filenames):
      with tf.gfile.GFile(filename, "rb") as f:
        image = f.read()
      print("Captions for image %s:" % os.path.basename(filenames[i]))
      num_in_data = 0
      for k in range(10):
          captions = generator.beam_search(sess, image)
            
          for i, caption in enumerate(captions):
            # Ignore begin and end words.
            sentence = [vocab.id_to_word(w) for w in caption.sentence[1:-1]]
            sentence = " ".join(sentence)
            in_data = 0
            if b_any(sentence in capt for capt in data_captions):
                in_data = 1
                num_in_data += 1
                num_in_data_total += 1
                num_captions += 1
            else:
                num_captions += 1
            print("  %d) %s (p=%f) [in data = %d]" % (i, sentence, math.exp(caption.logprob),in_data))
      print("number of captions in data = %d" % (num_in_data))
    print("(total number of captions in data = %d) percent in data = %f" % (num_in_data_total,(num_in_data_total/num_captions)))

['/Users/freedmand/scraps/danklearning/im2txt/Jmemes/sassy-gay-snape.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/romneyhood.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/ridiculously-photogenic-metalhead-guy.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/sad-dog.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/sudden-realization-ralph.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/stoner-stanley.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/stoner-dog.jpg', '/Users/freedmand/scraps/danklearning/im2txt/Jmemes/advice-dog.jpg']
INFO:tensorflow:Running caption generation on 8 files matching Jmemes
INFO:tensorflow:Loading model from checkpoint: trainlogIncNEW/model.ckpt
INFO:tensorflow:Restoring parameters from trainlogIncNEW/model.ckpt
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt
Captions for image sassy-gay-snape.jpg:
[[[5.8360700e-07 9.0317626e-05 1.6726404e-02 ... 5.6275796e-02
   4.1714955e-07 3.4112116e

  0) is this how i feel when i am (p=0.000000) [in data = 0]
  1) is this how i feel when i say we have a friend (p=0.000000) [in data = 0]
[[[5.8360700e-07 9.0317626e-05 1.6726404e-02 ... 5.6275796e-02
   4.1714955e-07 3.4112116e-05]
  [5.8360700e-07 9.0317626e-05 1.6726404e-02 ... 5.6275796e-02
   4.1714955e-07 3.4112116e-05]]]
[[[5.8360700e-07 9.0317626e-05 1.6726404e-02 ... 0.0000000e+00
   4.1714955e-07 3.4112116e-05]
  [5.8360700e-07 9.0317626e-05 1.6726404e-02 ... 5.6275796e-02
   4.1714955e-07 3.4112116e-05]]]
[[[3.5771311e-08 1.4245335e-03 2.7461024e-04 ... 1.2626706e-02
   4.2905718e-08 3.4391668e-04]
  [6.3919458e-08 4.3081993e-04 1.1177640e-04 ... 1.1591515e-02
   4.1302027e-08 2.0699807e-04]]]
[[[3.5771311e-08 1.4245335e-03 2.7461024e-04 ... 0.0000000e+00
   4.2905718e-08 3.4391668e-04]
  [6.3919458e-08 4.3081993e-04 1.1177640e-04 ... 1.1591515e-02
   4.1302027e-08 2.0699807e-04]]]
[[[9.4747953e-08 1.0872371e-03 2.3770927e-02 ... 1.0998554e-02
   1.2809267e-07 4.4338376e-0

[[[5.9447267e-08 1.1972853e-01 4.6011675e-03 ... 6.8587475e-03
   4.8849042e-08 4.2533699e-02]
  [6.2856600e-08 1.1517454e-03 1.4567935e-02 ... 1.0676949e-02
   4.6285138e-08 2.6281277e-04]]]
[[[5.9447267e-08 1.1972853e-01 4.6011675e-03 ... 0.0000000e+00
   4.8849042e-08 4.2533699e-02]
  [6.2856600e-08 1.1517454e-03 1.4567935e-02 ... 1.0676949e-02
   4.6285138e-08 2.6281277e-04]]]
[[[2.0825749e-07 1.4090406e-03 6.9282562e-02 ... 3.0289518e-02
   2.0535300e-07 1.4084985e-04]
  [2.2358295e-08 3.4298033e-01 4.8045837e-03 ... 4.0322370e-03
   2.2485313e-08 1.4907567e-02]]]
[[[2.0825749e-07 1.4090406e-03 6.9282562e-02 ... 0.0000000e+00
   2.0535300e-07 1.4084985e-04]
  [2.2358295e-08 3.4298033e-01 4.8045837e-03 ... 4.0322370e-03
   2.2485313e-08 1.4907567e-02]]]
[[[1.38409092e-07 1.96862407e-03 3.76500413e-02 ... 2.07060929e-02
   1.14832964e-07 1.33220339e-03]
  [8.75211157e-08 2.67883856e-03 7.50849023e-03 ... 7.78040476e-03
   6.39658921e-08 8.50749165e-02]]]
[[[1.38409092e-07 1.96862407

[[[2.5179804e-07 1.3855387e-03 4.8299883e-02 ... 2.1794248e-02
   2.0257930e-07 6.0734595e-04]
  [6.4059385e-08 3.0256358e-03 3.0124879e-02 ... 1.2672668e-02
   6.3150857e-08 5.3455372e-04]]]
[[[2.5179804e-07 1.3855387e-03 4.8299883e-02 ... 0.0000000e+00
   2.0257930e-07 6.0734595e-04]
  [6.4059385e-08 3.0256358e-03 3.0124879e-02 ... 1.2672668e-02
   6.3150857e-08 5.3455372e-04]]]
[[[2.4103082e-07 9.7635733e-05 1.4513150e-02 ... 1.7886618e-02
   2.0602580e-07 2.3659361e-04]
  [1.3555098e-07 1.8436490e-03 3.5036479e-03 ... 2.4592081e-02
   1.1628263e-07 2.7279074e-03]]]
[[[2.4103082e-07 9.7635733e-05 1.4513150e-02 ... 0.0000000e+00
   2.0602580e-07 2.3659361e-04]
  [1.3555098e-07 1.8436490e-03 3.5036479e-03 ... 2.4592081e-02
   1.1628263e-07 2.7279074e-03]]]
[[[3.0962113e-07 1.0609841e-04 2.8152007e-04 ... 3.8246952e-02
   3.9515359e-07 5.3854397e-04]
  [1.7167787e-08 9.2897011e-04 8.4730948e-04 ... 4.6678348e-03
   3.6868904e-08 3.3969167e-03]]]
[[[3.0962113e-07 1.0609841e-04 2.8152007

[[[2.0777227e-07 8.7880233e-04 1.7037563e-02 ... 5.2522399e-02
   1.1620829e-07 3.9918409e-03]
  [4.6393788e-07 1.1364610e-03 1.7186813e-02 ... 8.4389769e-02
   2.2437416e-07 5.8564977e-03]]]
[[[2.0777227e-07 8.7880233e-04 1.7037563e-02 ... 0.0000000e+00
   1.1620829e-07 3.9918409e-03]
  [4.6393788e-07 1.1364610e-03 1.7186813e-02 ... 8.4389769e-02
   2.2437416e-07 5.8564977e-03]]]
[[[1.7425236e-08 2.1736689e-02 2.0639366e-04 ... 5.0677466e-03
   1.3999810e-08 4.4091874e-01]
  [4.1963116e-08 9.0356534e-03 3.8649209e-04 ... 1.0209037e-02
   3.4356994e-08 5.7534158e-01]]]
[[[1.7425236e-08 2.1736689e-02 2.0639366e-04 ... 0.0000000e+00
   1.3999810e-08 4.4091874e-01]
  [4.1963116e-08 9.0356534e-03 3.8649209e-04 ... 1.0209037e-02
   3.4356994e-08 5.7534158e-01]]]
[[[1.04953401e-07 2.74955673e-04 6.46424131e-04 ... 2.80707255e-02
   9.10432050e-08 2.84524588e-03]
  [8.89429685e-08 1.24005936e-04 1.21214005e-04 ... 2.14109868e-02
   4.90484773e-08 3.86666768e-04]]]
[[[1.04953401e-07 2.74955673

...

[[[8.0479854e-09 1.9946141e-02 5.8386498e-04 ... 2.3666793e-03
   1.5518165e-08 3.3613053e-01]
  [2.5706035e-08 1.4950479e-02 1.4122388e-04 ... 6.4906599e-03
   5.7157109e-08 2.0566195e-01]]]
[[[8.0479854e-09 1.9946141e-02 5.8386498e-04 ... 0.0000000e+00
   1.5518165e-08 3.3613053e-01]
  [2.5706035e-08 1.4950479e-02 1.4122388e-04 ... 6.4906599e-03
   5.7157109e-08 2.0566195e-01]]]
[[[5.1984095e-08 3.7928517e-03 2.5925779e-01 ... 1.3237987e-02
   5.8013242e-08 1.9847913e-02]
  [2.0296932e-08 3.8749073e-04 7.1833550e-04 ... 2.3564233e-03
   1.9778483e-08 7.0078814e-01]]]
[[[5.1984095e-08 3.7928517e-03 2.5925779e-01 ... 0.0000000e+00
   5.8013242e-08 1.9847913e-02]
  [2.0296932e-08 3.8749073e-04 7.1833550e-04 ... 2.3564233e-03
   1.9778483e-08 7.0078814e-01]]]
[[[1.4554547e-07 1.9586529e-04 2.5349957e-04 ... 2.5189519e-02
   2.4206150e-07 4.7557880e-03]
  [1.9834476e-09 1.6324712e-04 2.1735468e-04 ... 6.3470902e-04
   2.8486076e-09 6.2476400e-02]]]
[[[1.4554547e-07 1.9586529e-04 2.5349957

[[[3.7486075e-08 1.8119391e-04 1.9377303e-04 ... 3.2925317e-03
   3.9265291e-08 2.7038605e-04]
  [6.9750726e-08 1.9961150e-04 1.1972534e-02 ... 1.3925052e-02
   4.0106801e-08 9.7322452e-05]]]
[[[3.7486075e-08 1.8119391e-04 1.9377303e-04 ... 0.0000000e+00
   3.9265291e-08 2.7038605e-04]
  [6.9750726e-08 1.9961150e-04 1.1972534e-02 ... 1.3925052e-02
   4.0106801e-08 9.7322452e-05]]]
  0) if i can't go to bed . . . . . . . . . not (p=0.000000) [in data = 0]
[[[5.8239345e-07 4.8922346e-05 7.6563018e-03 ... 3.9531890e-02
   2.6153248e-07 8.5187727e-05]
  [5.8239345e-07 4.8922346e-05 7.6563018e-03 ... 3.9531890e-02
   2.6153248e-07 8.5187727e-05]]]
[[[5.8239345e-07 4.8922346e-05 7.6563018e-03 ... 0.0000000e+00
   2.6153248e-07 8.5187727e-05]
  [5.8239345e-07 4.8922346e-05 7.6563018e-03 ... 3.9531890e-02
   2.6153248e-07 8.5187727e-05]]]
[[[6.7088315e-08 6.6973262e-02 8.0906721e-03 ... 2.7674939e-02
   1.4229711e-07 2.1483235e-03]
  [2.2172244e-07 4.0820963e-05 1.5786882e-02 ... 3.0512165e-02

[[[2.2172244e-07 4.0820963e-05 1.5786882e-02 ... 0.0000000e+00
   8.3544215e-08 6.4457476e-05]
  [1.9056408e-07 1.1028668e-03 7.4777001e-04 ... 7.8132808e-02
   1.9721134e-07 8.1767025e-04]]]
[[[1.1125820e-07 6.0395453e-02 1.3164113e-02 ... 2.4733918e-02
   6.7468250e-08 6.0261745e-02]
  [2.0529069e-07 4.6668906e-04 5.8039587e-02 ... 2.6114577e-02
   1.4939133e-07 3.0915192e-04]]]
[[[1.1125820e-07 6.0395453e-02 1.3164113e-02 ... 0.0000000e+00
   6.7468250e-08 6.0261745e-02]
  [2.0529069e-07 4.6668906e-04 5.8039587e-02 ... 2.6114577e-02
   1.4939133e-07 3.0915192e-04]]]
[[[5.1667121e-07 4.9994705e-04 3.4233805e-02 ... 5.5877075e-02
   1.2990456e-07 2.2636980e-04]
  [1.7536726e-07 1.3525771e-01 2.2957458e-03 ... 1.0556784e-02
   7.4912826e-08 2.4078960e-02]]]
[[[5.1667121e-07 4.9994705e-04 3.4233805e-02 ... 0.0000000e+00
   1.2990456e-07 2.2636980e-04]
  [1.7536726e-07 1.3525771e-01 2.2957458e-03 ... 1.0556784e-02
   7.4912826e-08 2.4078960e-02]]]
[[[1.0544149e-07 3.7518246e-04 2.8923681

[[[3.3125804e-07 3.8490172e-03 9.7690068e-02 ... 3.5057884e-02
   1.3495325e-07 2.5085534e-03]
  [1.2982214e-07 9.4924857e-05 1.1130163e-02 ... 1.4948818e-02
   8.4612090e-08 1.4687933e-04]]]
[[[3.3125804e-07 3.8490172e-03 9.7690068e-02 ... 0.0000000e+00
   1.3495325e-07 2.5085534e-03]
  [1.2982214e-07 9.4924857e-05 1.1130163e-02 ... 1.4948818e-02
   8.4612090e-08 1.4687933e-04]]]
[[[9.8666064e-08 5.0403987e-04 8.2834132e-02 ... 1.2389061e-02
   5.7085789e-08 1.7418364e-03]
  [6.1545663e-08 3.4482286e-03 6.0789499e-02 ... 7.2194515e-03
   4.1861124e-08 4.1504698e-03]]]
[[[9.8666064e-08 5.0403987e-04 8.2834132e-02 ... 0.0000000e+00
   5.7085789e-08 1.7418364e-03]
  [6.1545663e-08 3.4482286e-03 6.0789499e-02 ... 7.2194515e-03
   4.1861124e-08 4.1504698e-03]]]
[[[1.17219805e-07 1.41786709e-02 2.80177891e-02 ... 9.28908866e-03
   3.84635150e-08 1.50434962e-02]
  [3.89568271e-07 3.33979260e-04 8.49235454e-04 ... 3.87412608e-02
   2.01895944e-07 8.26109375e-04]]]
[[[1.17219805e-07 1.41786709

[[[3.1155025e-08 2.2155479e-02 5.9301197e-04 ... 4.7325213e-03
   3.2220534e-08 1.5715091e-01]
  [4.1661150e-07 2.6511142e-04 1.2085564e-04 ... 6.3520558e-02
   1.9958169e-07 1.1523496e-03]]]
[[[3.1155025e-08 2.2155479e-02 5.9301197e-04 ... 0.0000000e+00
   3.2220534e-08 1.5715091e-01]
  [4.1661150e-07 2.6511142e-04 1.2085564e-04 ... 6.3520558e-02
   1.9958169e-07 1.1523496e-03]]]
[[[5.5163810e-08 8.3098868e-03 1.9842833e-04 ... 1.1154705e-02
   6.5912467e-08 1.6172269e-01]
  [1.7503389e-08 1.1029106e-02 3.6224106e-04 ... 2.9227650e-03
   1.1285527e-08 1.9050001e-01]]]
[[[5.5163810e-08 8.3098868e-03 1.9842833e-04 ... 0.0000000e+00
   6.5912467e-08 1.6172269e-01]
  [1.7503389e-08 1.1029106e-02 3.6224106e-04 ... 2.9227650e-03
   1.1285527e-08 1.9050001e-01]]]
[[[2.0224032e-08 9.5062004e-03 3.8194363e-03 ... 3.1169020e-03
   1.9769038e-08 1.8152420e-01]
  [7.3137848e-08 6.3063204e-04 4.8381463e-03 ... 2.1887042e-02
   8.3140044e-08 1.8783993e-01]]]
[[[2.0224032e-08 9.5062004e-03 3.8194363

In [21]:
np.random.seed(2)
seq_embeddings = np.ones([1, 2, 300])
state_feed = np.ones([1, 2, 1024])
with tf.Session(graph=g) as sess:
    # Load the model from checkpoint.
    restore_fn(sess)
    softmax_output, state_output = sess.run(
        fetches=["softmax:0", "lstm/state:0"],
        feed_dict={
            #"input_feed:0": input_feed,
            "lstm/state_feed:0": state_feed,
            "seq_embeddings:0": seq_embeddings,
            #"seq_embedding/embedding_map:0": self.embedding_map
        })
    print(state_output.shape)
    print(softmax_output.shape)

INFO:tensorflow:Loading model from checkpoint: trainlogIncNEW/model.ckpt
INFO:tensorflow:Restoring parameters from trainlogIncNEW/model.ckpt
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt
(1, 2, 1024)
(1, 2, 38521)


In [6]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1000px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [7]:
show_graph(g)

In [8]:
OLD_CHECKPOINT_FILE = "trainlogSGD/model.ckpt-600000"
NEW_CHECKPOINT_FILE = "trainlogSGDNEW/model.ckpt-600000"

import tensorflow as tf
vars_to_rename = {
    "lstm/basic_lstm_cell/weights": "lstm/basic_lstm_cell/kernel",
    "lstm/basic_lstm_cell/biases": "lstm/basic_lstm_cell/bias",
}
new_checkpoint_vars = {}
reader = tf.train.NewCheckpointReader(OLD_CHECKPOINT_FILE)
for old_name in reader.get_variable_to_shape_map():
  if old_name in vars_to_rename:
    new_name = vars_to_rename[old_name]
  else:
    new_name = old_name
  new_checkpoint_vars[new_name] = tf.Variable(reader.get_tensor(old_name))

init = tf.global_variables_initializer()
saver = tf.train.Saver(new_checkpoint_vars)

with tf.Session() as sess:
  sess.run(init)
  saver.save(sess, NEW_CHECKPOINT_FILE)

InvalidArgumentError: Unsuccessful TensorSliceReader constructor: Failed to get matching files on trainlogSGD/model.ckpt-600000: Not found: trainlogSGD; No such file or directory